In [ ]:
import os
import shutil

from dateutil.parser import parse
import django
import pandas as pd

os.environ.setdefault("DJANGO_SETTINGS_MODULE", "mastspec.settings")
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

django.setup()

from plotter.models import *
# from mastspec.views import *
# from mastspec.forms import *
from utils import rows, columns

In [114]:
os.getcwd()

'/home/michael/scratch/python-scratch/mastspec/mastspec'

In [117]:
S_PATH = "/home/michael/Downloads/MSP_sample_data/"
BASE_DIR = os.getcwd()
IMAGE_PATH = os.path.join(BASE_DIR, "static_in_pro/our_static/img")

In [3]:
metaframe = pd.read_csv(S_PATH+'Metadata.csv')\
    .dropna()\
    .drop(columns=['Focal Distance.1'])\

metaframe.columns=['sol','mcam','name','rover_el','target_el','tau',
              'ltst','f_dist','i_angle','e_angle','so_lon','site',
              'drive','lat','lon','traverse']


,sol,mcam,name,rover_el,target_el,tau,ltst,f_dist,i_angle,e_angle,so_lon,site,drive,lat,lon,traverse
0,2029.0,mcam10709,Taconite Stereo,-4165.89,#NAME?,0.637762,11:46:22,0.960,12.1799,73.6722,163.166667,69.0,2456.0,-4.724632,137.38017,19000.284
1,2034.0,mcam10737,Logan,-4168.18,#NAME?,0.675107,12:40:55,18.610,14.7135,75.8957,165.900000,69.0,2766.0,-4.724634,137.37966,19034.741
2,2036.0,mcam10747,Britt Stereo,-4175.97,-4176.3494,0.644188,12:11:25,5.187,10.5050,63.9312,167.500000,70.0,0.0,-4.724193,137.37950,19063.505
3,2036.0,mcam10749,Aurora Stereo,-4175.97,Infinity,0.644188,12:18:04,0.140,11.0759,91.4793,167.500000,70.0,0.0,-4.724193,137.37950,19063.505
4,2038.0,mcam10762,Red Cliff 2x1,-4174.72,#NAME?,0.566180,14:14:55,0.130,35.0700,81.5508,168.233333,70.0,240.0,-4.724271,137.37952,0.000
5,2041.0,mcam10786,Barto Lake & Homer Lake,-4178.79,-4179.5044,0.512000,12:32:19,4.444,12.0588,53.9630,169.700000,70.0,886.0,-4.724773,137.37809,0.000
6,2043.0,mcam10810,Pigeon River & Bald Eagle Lake 2x1,-4179.22,-4179.9381,0.535079,11:31:18,3.373,11.0705,39.0878,170.766667,70.0,1000.0,-4.724746,137.37784,0.000
7,2046.0,mcam10829,Mud Lake Stereo,-4180.64,-4181.2176,0.663340,13:01:35,3.760,17.2495,48.7805,172.450000,70.0,1430.0,-4.724832,137.37742,19215.802
8,2050.0,mcam10863,Giants Range and Bilbert,-4187.41,-4187.2963,0.640301,12:18:26,3.438,8.2164,58.6962,174.650000,70.0,1538.0,-4.724478,137.37738,19253.808
9,2050.0,mcam10865,Floodwood DRT Stereo,-4187.41,-4187.7261,0.640301,12:25:51,3.647,9.3863,52.5796,174.650000,70.0,1538.0,-4.724478,137.37738,19253.808


In [30]:
metaframe.iloc[8].columns

AttributeError: 'Series' object has no attribute 'columns'

In [4]:
for obs in rows(metaframe):
    obs = MObs(**dict(zip(obs.index,obs.values)))
    obs.ltst = parse(obs.ltst).time()
    if obs.target_el == '#NAME?':
        obs.target_el = None
    elif obs.target_el == 'Infinity':
        obs.target_el = float('inf')
    obs.save()

In [6]:
MObs.objects.all()

<QuerySet [<MObs: MObs object (1)>, <MObs: MObs object (2)>, <MObs: MObs object (3)>, <MObs: MObs object (4)>, <MObs: MObs object (5)>, <MObs: MObs object (6)>, <MObs: MObs object (7)>, <MObs: MObs object (8)>, <MObs: MObs object (9)>, <MObs: MObs object (10)>, <MObs: MObs object (11)>, <MObs: MObs object (12)>, <MObs: MObs object (13)>, <MObs: MObs object (14)>, <MObs: MObs object (15)>, <MObs: MObs object (16)>, <MObs: MObs object (17)>, <MObs: MObs object (18)>, <MObs: MObs object (19)>, <MObs: MObs object (20)>]>

In [99]:
# hopefully this is consistent! i think so; it's just sorted alphanumeric increasing
# without accounting for magnitude. if not, can make it more sophisticated

filter_index = [
    'L6', 'R6', 'L2', 'R2', 'R0_blue', 'L0_blue', 'L1', 
    'R1', 'R0_green', 'L0_green', 'R0_red', 'L0_red',
    'L4', 'L3', 'R3', 'L5', 'R4', 'R5'
]

In [102]:
os.listdir(directory)

['Working']

In [96]:
frame['Float']

 red Mean Value                    NaN
 red Standard Deviation            NaN
 light green Mean Value            NaN
 light green Standard Deviation    NaN
 light blue Mean Value             NaN
 light blue Standard Deviation     NaN
 light cyan Mean Value             NaN
 light cyan Standard Deviation     NaN
 teal Mean Value                   NaN
 teal Standard Deviation           NaN
 dark blue Mean Value              NaN
 dark blue Standard Deviation      NaN
Name: Float, dtype: object

In [118]:
directories = [S_PATH+file for file in os.listdir(S_PATH) if os.path.isdir(S_PATH+file)]

for directory in directories:
    mcam = directory[directory.find('mcam'):]
    obs = MObs.objects.get(mcam__iexact=mcam)
    # sometimes the directory contains /working; sometimes /Working
    path = directory+'/'+os.listdir(directory)[0]+'/'
    contents = os.listdir(path) 
    spec_files = [file for file in contents if file[-3:] == 'csv']
    for spec_file in spec_files:
        frame = pd.read_csv(path+spec_file).T
        frame = frame.drop(['# Wavelength (nm)', ' Eye'])
        columns = filter_index + list(rows(frame)[0].values[18:])
        frame.columns = columns
        index = []
        for n in range(len(frame.index)):
            line = frame.iloc[n]
            if 'Mean' not in line.name:
                continue
            stdev_line = frame.iloc[n+1]
            color_end = line.name.find('Mean')
            color = line.name[1:color_end-1]
            spectrum = MSpec()
            if 'R2' in spec_file:
                spectrum.image_number = 2
            spectrum.observation = obs
            spectrum.roi_color = color
            for filt in filter_index:
                setattr(spectrum,filt+'_mean',float(line[filt]))
                setattr(spectrum,filt+'_stdev',float(stdev_line[filt]))
            if 'Float' in frame.columns:
                if line['Float']:
                    spectrum.is_floating = True
            for column_name in ['Notes', 'Group', 'Formation', 'Member']:
                if column_name in frame.columns:
                    setattr(spectrum,column_name.lower(),line[column_name])
            spectrum.save()
    image_files = [file for file in contents if file[-3:] in ['png','jpg']] 
    for file in image_files:
        shutil.copy(path+file, IMAGE_PATH+file)
        if 'ROIs_OVERLAY' in file:
            if 'L126' in file:
                if 'R2' in file:
                    obs.lefteye_s_image_2 = IMAGE_PATH+file
                else:
                    obs.lefteye_s_image_1 = IMAGE_PATH+file
            elif 'R126' in file:
                if 'R2' in file:
                    obs.righteye_s_image_2 = IMAGE_PATH+file
                else:
                    obs.righteye_s_image_1 = IMAGE_PATH+file
            elif 'RGB':
                if 'L_' in file:
                    obs.lefteye_rgb_image_1 = IMAGE_PATH+file
                elif 'R1' in file:
                    obs.righteye_rgb_image_1 = IMAGE_PATH+file
                elif 'R2' in file:
                    obs.righteye_rgb_image_2 = IMAGE_PATH+file
    obs.save()

In [ ]:
MObs.object